In [1]:
!python -V

Python 3.11.4


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///c:/Users/bylsm/jbyls-git/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1717332736728, experiment_id='1', last_update_time=1717332736728, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715205396466

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "jason")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [14]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [15]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:51:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.55218                           
[1]	validation-rmse:8.09182                           
[2]	validation-rmse:7.33495                           
[3]	validation-rmse:6.94579                           
[4]	validation-rmse:6.74877                           
[5]	validation-rmse:6.64175                           
[6]	validation-rmse:6.57993                           
[7]	validation-rmse:6.54377                           
[8]	validation-rmse:6.52120                           
[9]	validation-rmse:6.50297                           
[10]	validation-rmse:6.49321                          
[11]	validation-rmse:6.48415                          
[12]	validation-rmse:6.47666                          
[13]	validation-rmse:6.47294                          
[14]	validation-rmse:6.46919                          
[15]	validation-rmse:6.46491                          
[16]	validation-rmse:6.46149                          
[17]	validation-rmse:6.45923                          
[18]	valid

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:51:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.30539                                                   
[1]	validation-rmse:9.00526                                                    
[2]	validation-rmse:8.14179                                                    
[3]	validation-rmse:7.58206                                                    
[4]	validation-rmse:7.22319                                                    
[5]	validation-rmse:6.99324                                                    
[6]	validation-rmse:6.84643                                                    
[7]	validation-rmse:6.74880                                                    
[8]	validation-rmse:6.68517                                                    
[9]	validation-rmse:6.63829                                                    
[10]	validation-rmse:6.60684                                                   
[11]	validation-rmse:6.58429                                                   
[12]	validation-rmse:6.56470            

...

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:00:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.07277                                                    
[1]	validation-rmse:10.14435                                                    
[2]	validation-rmse:9.39681                                                     
[3]	validation-rmse:8.79735                                                     
[4]	validation-rmse:8.32196                                                     
[5]	validation-rmse:7.94577                                                     
[6]	validation-rmse:7.65158                                                     
[7]	validation-rmse:7.42259                                                     
[8]	validation-rmse:7.24440                                                     
[9]	validation-rmse:7.10498                                                     
[10]	validation-rmse:6.99509                                                    
[11]	validation-rmse:6.90856                                                    
[12]	validation-rmse:6.84153

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:01:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



 32%|███▏      | 16/50 [10:17<21:53, 38.62s/trial, best loss: 6.324251761041095]


KeyboardInterrupt: 

In [16]:
mlflow.xgboost.autolog(disable=True)

In [17]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09078888650095844,
        'max_depth': 52,
        'min_child_weight': 1.4267910021048467,
        'objective': 'reg:linear',
        'reg_alpha': 0.006826686440337911,
        'reg_lambda': 0.3269539352340935,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:02:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.47551
[1]	validation-rmse:10.82367
[2]	validation-rmse:10.25219
[3]	validation-rmse:9.75045
[4]	validation-rmse:9.31230
[5]	validation-rmse:8.92849
[6]	validation-rmse:8.59532
[7]	validation-rmse:8.30794
[8]	validation-rmse:8.05858
[9]	validation-rmse:7.84627
[10]	validation-rmse:7.66071
[11]	validation-rmse:7.50329
[12]	validation-rmse:7.36531
[13]	validation-rmse:7.24836
[14]	validation-rmse:7.14523
[15]	validation-rmse:7.05904
[16]	validation-rmse:6.98431
[17]	validation-rmse:6.91936
[18]	validation-rmse:6.86319
[19]	validation-rmse:6.81534
[20]	validation-rmse:6.77338
[21]	validation-rmse:6.73754
[22]	validation-rmse:6.70553
[23]	validation-rmse:6.67799
[24]	validation-rmse:6.65361
[25]	validation-rmse:6.63093
[26]	validation-rmse:6.61239
[27]	validation-rmse:6.59523
[28]	validation-rmse:6.58014
[29]	validation-rmse:6.56627
[30]	validation-rmse:6.55434
[31]	validation-rmse:6.54380
[32]	validation-rmse:6.53419
[33]	validation-rmse:6.52569
[34]	validation-rmse:

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:03:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [19]:
logged_model = 'runs:/831c81814047441fa84d781a0d73212f/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:00:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [20]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 831c81814047441fa84d781a0d73212f

In [21]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

c:\Users\bylsm\jbyls-git\mlops-zoomcamp\02-experiment-tracking\.env\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:01:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [22]:
xgboost_model

In [23]:
y_ored = xgboost_model.predict(valid)

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
